## Notebook to look at trends in cell density

In [1]:
import os
import pandas as pd
import numpy as np
from src.utilities.shape_utils import plot_mesh
from scipy.spatial import KDTree
import plotly.express as px
import trimesh

# Load test dataset that used Kikume NLS marker
root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\killi_tracker\\"
experiment_date = "20240611_NLS-Kikume_24hpf_side2"
config_name = "tracking_jordao_20240918.txt"
model ="LCP-Multiset-v1"
tracking_folder = config_name.replace(".txt", "")
tracking_folder = tracking_folder.replace(".toml", "")

well_num = 0
start_i = 0
stop_i = 1600

suffix = ""

# get path to metadata
metadata_path = os.path.join(root, "metadata", "tracking")

# set output path for tracking results
project_path = os.path.join(root, "tracking", experiment_date,  tracking_folder, f"well{well_num:04}" + suffix, "")
project_sub_path = os.path.join(project_path, f"track_{start_i:04}" + f"_{stop_i:04}" + suffix, "")

# load the tracks 
deep_tracks_df = pd.read_csv(os.path.join(project_sub_path, "deep_tracks_df.csv"))

# load velocity info
cell_density_df = pd.read_csv(os.path.join(project_sub_path, "cell_density_df2.csv"))

# load sphere mesh
sphere_mesh = trimesh.load(os.path.join(project_sub_path, "embryo_sphere_mesh.obj"))

### Calculate velocity magnitude at vertices of a lower-res sphere

In [19]:
# smooth
f = sphere_mesh.faces.copy()
v = sphere_mesh.vertices.copy()

# get moving average for each face over time
face_cols = [f"density_f{i:05}" for i in range(f.shape[0])]

avg_window = 15
cell_density_df_sm = cell_density_df.copy()
cell_density_df_sm[face_cols] = cell_density_df_sm[face_cols].rolling(center=True, window=avg_window, min_periods=1).mean()
cell_density_df_sm.head()

time  baseline  density_f00000  density_f00001  density_f00002  \
0   0.0  0.004725             0.0             0.0             0.0   
1   1.0  0.004694             0.0             0.0             0.0   
2   2.0  0.004675             0.0             0.0             0.0   
3   3.0  0.004651             0.0             0.0             0.0   
4   4.0  0.004630             0.0             0.0             0.0   

   density_f00003  density_f00004  density_f00005  density_f00006  \
0             0.0             0.0             0.0             0.0   
1             0.0             0.0             0.0             0.0   
2             0.0             0.0             0.0             0.0   
3             0.0             0.0             0.0             0.0   
4             0.0             0.0             0.0             0.0   

   density_f00007  ...  density_f20470  density_f20471  density_f20472  \
0             0.0  ...             0.0             0.0             0.0   
1             0.0  ...             0.0             0.0             0.0   
2             0.0  ...             0.0             0.0             0.0   
3             0.0  ...             0.0             0.0             0.0   
4             0.0  ...             0.0             0.0             0.0   

   density_f20473  density_f20474  density_f20475  density_f20476  \
0             0.0             0.0             0.0             0.0   
1             0.0             0.0             0.0             0.0   
2             0.0             0.0             0.0             0.0   
3             0.0             0.0             0.0             0.0   
4             0.0             0.0             0.0             0.0   

   density_f20477  density_f20478  density_f20479  
0             0.0             0.0             0.0  
1             0.0             0.0             0.0  
2             0.0             0.0             0.0  
3             0.0             0.0             0.0  
4             0.0             0.0             0.0  

[5 rows x 20482 columns]

In [23]:
import plotly.graph_objects as go
from tqdm import tqdm
from src.utilities.plot_utils import mesh_face_plot
from scipy.interpolate import griddata

# smooth
f = sphere_mesh.faces.copy()
v = sphere_mesh.vertices.copy()

cl = -0.005 # + 0.04#+ density_baseline
cu = 0.006 #+ 0.04# + density_baseline
frame_path = os.path.join(project_sub_path, "fig00_density_frames_v3", "")
os.makedirs(frame_path, exist_ok=True)

time_vec = cell_density_df_sm["time"].to_numpy()
baseline_vec = cell_density_df_sm["baseline"].to_numpy()

plot_interval = 5

for t in tqdm(range(0, int(time_vec[-1]), plot_interval)):
    
#     t = 1200
    
    plot_time = int(time_vec[t])

    # for t, plot_time in enumerate(tqdm(sh_time_vec)):
    d_raw = cell_density_df_sm.loc[plot_time, face_cols] 
    nz_filter = d_raw > 0
    d_mu = np.mean(d_raw[nz_filter])
    d_vec = d_raw - d_mu
    # dl = np.min(d_norm)
    # dl_factor = cl / dl * 0.8
    # d_vec = d_norm.copy()
    # # rescale
    # d_vec[d_vec < 0] = d_vec[d_vec < 0]*dl_factor


    camera=dict(
                eye=dict(x=1, y=1, z=1),  # Camera position
                up=dict(x=0, y=0, z=1),     # "Up" direction
    #             center=dict(x=0, y=0, z=0)  # Focal point
            )

    # Extract x, y, z coordinates from vertices
#     x, y, z = v[:, 0], v[:, 1], v[:, 2]

#     # Extract i, j, k indices from faces
#     i, j, k = f[:, 0], f[:, 1], f[:, 2]

    fig = mesh_face_plot(f, v, d_vec, colormap="RdBu_r") #"deep_r")
#     fig = go.Figure(data=[go.Mesh3d(
#             x=x, y=y, z=z,            # Vertex coordinates
#             i=i, j=j, k=k,
#             intensity=d_vec,
#             flatshading=True,
#             colorscale="RdBu_r"
#         )])

    fig.update_layout(
        scene=dict(camera=dict(
                eye=dict(x=0, y=0.35, z=2),  # Camera position
                up=dict(x=0, y=-1, z=0),     # "Up" direction
                center=dict(x=0, y=0, z=0)  # Focal point
            ),
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    fig.update_traces(lightposition=dict(
            x=5,  # Position the light source
            y=5,
            z=5
        ), 
            lighting=dict(
            ambient=0.85,    # Soft overall lighting
            diffuse=0.2,    # Scattered light
            specular=0,   # Shiny reflections
            roughness=0.2,  # Smooth surface
            fresnel=0     # Glow at edges
        ),

                    )

    fig.update_layout(
    height=800,  # Overall figure height
    width=800,  # Overall figure width
    )
    
    fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False),  # Hide x-axis
        yaxis=dict(visible=False),  # Hide y-axis
        zaxis=dict(visible=False),  # Hide z-axis
    )
    )

    fig.update_traces(cmin=cl, cmax=cu, showscale=False)
#     fig.update_traces(colorbar=dict(title=dict(
#                 text="density",  # Add LaTeX-style formula
#                 font=dict(size=14),
#                 side="top"# Customize font size
#             )))
    # fig.show()
    fig.write_image(frame_path + f"density_frame{int(plot_time):04}.png", scale=1)

100%|██████████| 320/320 [16:43<00:00,  3.14s/it]


In [21]:
np.max(d_vec)

0.00333524297387779